In [1]:
import sys
import numpy as np
import cupy as cp

sys.path.append('../')

from mandala.nodecore import Node
from mandala.nodecore import Variable
from mandala.autodiff import autodiff
from mandala.autodiff.linear import Linear
from mandala.autodiff import initializers
from mandala import cuda

In [2]:
xp = cp

In [3]:
import cupy
from mandala import cuda
from mandala.nodecore import Node
from mandala.autodiff import autodiff


if cuda.cudnn_enable:
    cudnn = cuda.cudnn
    _mode = cupy.cuda.cudnn.CUDNN_ACTIVATION_RELU


def forward_relu(x):
    xp = cuda.get_array_module(x)

    if xp == cupy and cuda.cudnn_enable:
         y = cudnn.activation_forward(x, _mode)
    else:
        y = xp.maximum(x, 0)

    return y


def backward_relu(x, y, gy):
    xp = cuda.get_array_module(x)

    if xp == cupy and cuda.cudnn_enable:
        gx = cudnn.activation_backward(x, y, gy, _mode)
    else:
        gx = (x > 0) * gy

    return gx


class ReLUFunction(autodiff.AutoDiff):

    def forward(self, xs):
        x, = xs
        self.y = Node(forward_relu, [x])
        return self.y
    
    def backward(self, xs, gy):
        x, = xs
        gx = Node(backward_relu, [x, self.y, gy])
        return gx,


def relu(x):
    return ReLUFunction()([x])

In [4]:
xp == cupy

True

In [5]:
x = np.arange(-5, 5, dtype=np.float32)
x = Variable(x)

In [6]:
x.data

array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.], dtype=float32)

In [7]:
x.to_gpu()

In [8]:
y = relu(x)

In [9]:
y.data

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  3.,  4.], dtype=float32)

In [10]:
y.backward(gy=xp.ones_like(y.data))

In [11]:
x.grad.data

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.], dtype=float32)